# Quantum Monte Carlo algorithms 

In [7]:
import random
import matplotlib.pyplot as plt
import math
from qiskit import QuantumCircuit, QuantumRegister, ClassicalRegister, transpile
from qiskit_aer import Aer
from qiskit.circuit.library import RYGate, QFT, GroverOperator, GlobalPhaseGate
from qiskit.quantum_info import Operator 
from qiskit.primitives import Sampler
from qiskit_algorithms import AmplitudeEstimation, EstimationProblem
import numpy as np

Estimation of the mean with bounded L2 norm

In [51]:
# number of qubits
n = 3 

# generate output values in [0, 4]
output_values = [random.uniform(0, 4) for _ in range(2**n)]
print(output_values)

[2.6216862761593824, 2.194321328722708, 3.156277587872592, 1.5565770938006556, 1.2310172850857932, 0.02240434195214469, 1.8682511080679438, 2.28929856159165]


In [52]:
expectation = sum(output_values)/len(output_values)
print(f'Expectation value: {expectation}')

Expectation value: 1.8674791979066088


In [53]:
# create mapping
bit_strings = [format(i, f'0{n}b') for i in range(2**n)]
mapping = dict(zip(bit_strings, output_values))
mapping

{'000': 2.6216862761593824,
 '001': 2.194321328722708,
 '010': 3.156277587872592,
 '011': 1.5565770938006556,
 '100': 1.2310172850857932,
 '101': 0.02240434195214469,
 '110': 1.8682511080679438,
 '111': 2.28929856159165}

Define functions

In [20]:
def A_1(value, x):
    """
    Returns the value if it is less than x, otherwise return 0.
    """
    if value < x:
        res = value
    else:
        res = 0 
    return res

def A_2(value, x, y):
    """
    Returns the value if it is between x and y 
    (inclusive of x, exclusive of y), otherwise return 0.
    """
    if value >= x and value < y:
        res = value
    else:
        res = 0 
    return res

def A_3(value, y):
    """
    Returns the value if it is greater than or equal to y, 
    otherwise return 0.
    """ 
    if value >= y:
        res = value
    else:
        res = 0 
    return res

def A_prime(value, sigma):
    return value/sigma

def state_function(state, value, qc):
    """
    Applies controlled rotations based on the state and value.

    Parameters:
    state (str): The bit string representing the state.
    value (float): The value for rotation.
    qc (QuantumCircuit): The quantum circuit.
    n (int): Number of qubits per time step.
    k (int): Number of time steps.
    """
    
    for i in range(len(state)):
        if state[i] == '0':
            qc.x([len(state)-i])

    # calculate the rotation angle
    theta = math.asin(value**0.5)*2   

    # create a controlled RY gate
    rot = RYGate(theta).control(len(state))

    # apply the controlled rotation 
    qc.append(rot, [1, 2, 3, 0])
    
    for i in range(len(state)):
        if state[i] == '0':
            qc.x([len(state)-i]) 

def V_operator(qc):
    """
    Applies the phase oracle (V operator) to the quantum circuit.

    Parameters:
    qc (QuantumCircuit): The quantum circuit.
    """
    qc.h([0])
    qc.x([0])
    qc.h([0])

Create phase oracle

In [6]:
# create oracle
qr = QuantumRegister(n+1)
oracle = QuantumCircuit(qr)
V_operator(oracle)
oracle.draw()

┌───┐┌───┐┌───┐
q1_0: ┤ H ├┤ X ├┤ H ├
      └───┘└───┘└───┘
q1_1: ───────────────
                     
q1_2: ───────────────
                     
q1_3: ───────────────

Run algorithm

In [54]:
m = 6 # ancilla qubits 
j = int(np.ceil(np.log2(max(mapping.values())))) # max intervals

mu_estimation = 0
temp = []

for value in mapping.values():
    temp.append(A_2(value, 0, 1))

# create state preperation
qr = QuantumRegister(n+1)
state_prearation = QuantumCircuit(qr)

# create uniform probability distribution
state_prearation.h([x+1 for x in range(n)]) 

# apply state function
for i, amplitude in enumerate(temp):
    state_function(state=bit_strings[i], value=amplitude, qc=state_prearation)

Q = GroverOperator(oracle=oracle, 
                   state_preparation=state_prearation, 
                   zero_reflection=None, 
                   reflection_qubits=None, 
                   insert_barriers=False, 
                   mcx_mode='noancilla', 
                   name='Q')

sampler = Sampler()
ae = AmplitudeEstimation(
    num_eval_qubits=m,  # the number of evaluation qubits specifies circuit width and accuracy
    sampler=sampler,
)

problem = EstimationProblem(
    state_preparation=state_prearation,  # A operator
    grover_operator=Q,  # Q operator
    objective_qubits=[0],  # the "good" state Psi1 is identified as measuring |1> in qubit 0
)

ae_result = ae.estimate(problem)
mu_estimation += ae_result.estimation

for i in range(j):
    temp = []
    for value in mapping.values():
        temp.append(A_2(value, 2**i, 2**(i+1))/2**(i+1))

    # create state preperation
    qr = QuantumRegister(n+1)
    state_prearation = QuantumCircuit(qr)

    # create uniform probability distribution
    state_prearation.h([x+1 for x in range(n)]) 

    # apply state function
    for x, amplitude in enumerate(temp):
        state_function(state=bit_strings[x], value=amplitude, qc=state_prearation)

    Q = GroverOperator(oracle=oracle, 
                       state_preparation=state_prearation, 
                       zero_reflection=None, 
                       reflection_qubits=None, 
                       insert_barriers=False, 
                       mcx_mode='noancilla', 
                       name='Q')

    sampler = Sampler()
    ae = AmplitudeEstimation(
        num_eval_qubits=m,  # the number of evaluation qubits specifies circuit width and accuracy
        sampler=sampler,
    )

    problem = EstimationProblem(
        state_preparation=state_prearation,  # A operator
        grover_operator=Q,  # Q operator
        objective_qubits=[0],  # the "good" state Psi1 is identified as measuring |1> in qubit 0
    )

    ae_result = ae.estimate(problem)
    mu_estimation += ae_result.estimation*2**(i+1)

Estimation with bouded variance

In [18]:
# number of qubits
n = 3 

# generate random output values (both positive and negative values)
output_values = [random.uniform(-10, 20) for _ in range(2**n)]

In [19]:
# map domain values to bits
bit_strings = [format(i, f'0{n}b') for i in range(2**n)]
mapping = dict(zip(bit_strings, output_values))
mapping

{'000': 6.93025441441646,
 '001': 17.510738923046564,
 '010': -4.704285917229183,
 '011': -3.122115798698405,
 '100': 7.87914191159631,
 '101': 9.95132697516685,
 '110': -7.950494461845844,
 '111': -7.049348036694672}

In [22]:
# compute the variance and mean

mu = sum(output_values)/len(output_values)
var = 0

for val in output_values:
    var += ((val-mu)**2)

var = var/len(output_values)

print(mu,var)

76.66015122004146

Run algorithm

In [35]:
value = random.choice(output_values) # generate random value
mean_proxy = A_prime(value, var**0.5) # find a mean proxy 
new_output_values = [A_prime(value, var**0.5)-mean_proxy for value in output_values] # scale output values
m = 8 # ancilla qubits 

temp = []

for value in new_output_values:
    temp.append(-A_1(value, 0)/4)

# create state preperation
qr = QuantumRegister(n+1)
state_prearation = QuantumCircuit(qr)

# create uniform probability distribution
state_prearation.h([x+1 for x in range(n)]) 

# apply state function
for i, amplitude in enumerate(temp):
    state_function(state=bit_strings[i], value=amplitude, qc=state_prearation)

Q = GroverOperator(oracle=oracle, 
                   state_preparation=state_prearation, 
                   zero_reflection=None, 
                   reflection_qubits=None, 
                   insert_barriers=False, 
                   mcx_mode='noancilla', 
                   name='Q'
                   )

sampler = Sampler()
ae = AmplitudeEstimation(
    num_eval_qubits=m,  # the number of evaluation qubits specifies circuit width and accuracy
    sampler=sampler,
)

problem = EstimationProblem(
    state_preparation=state_prearation,  # A operator
    grover_operator=Q,  # Q operator
    objective_qubits=[0],  # the "good" state Psi1 is identified as measuring |1> in qubit 0
)

ae_result = ae.estimate(problem)
mu_negative = ae_result.estimation


temp = []

for value in new_output_values:
    temp.append(A_3(value, 0)/4)

# create state preperation
qr = QuantumRegister(n+1)
state_prearation = QuantumCircuit(qr)

# create uniform probability distribution
state_prearation.h([x+1 for x in range(n)]) 

# apply state function
for i, amplitude in enumerate(temp):
    state_function(state=bit_strings[i], value=amplitude, qc=state_prearation)

Q = GroverOperator(oracle=oracle, 
                   state_preparation=state_prearation, 
                   zero_reflection=None, 
                   reflection_qubits=None, 
                   insert_barriers=False, 
                   mcx_mode='noancilla', 
                   name='Q')

sampler = Sampler()
ae = AmplitudeEstimation(
    num_eval_qubits=m,  # the number of evaluation qubits specifies circuit width and accuracy
    sampler=sampler,
)

problem = EstimationProblem(
    state_preparation=state_prearation,  # A operator
    grover_operator=Q,  # Q operator
    objective_qubits=[0],  # the "good" state Psi1 is identified as measuring |1> in qubit 0
)

ae_result = ae.estimate(problem)
mu_positive = ae_result.estimation

mu = mean_proxy - 4*mu_negative + 4*mu_positive
mu = mu*var**0.5
print(mu)

2.569001349790952


Estimation with bounded relative error

In [59]:
# number of qubits
n = 3

# generate random possible output values
output_values = [random.uniform(0, 20) for _ in range(2**n)]

In [60]:
# map output values to bits
bit_strings = [format(i, f'0{n}b') for i in range(2**n)]
mapping = dict(zip(bit_strings, output_values))
mapping

{'000': 7.00820470556669,
 '001': 19.19671010598729,
 '010': 11.591331976796607,
 '011': 17.560546899686223,
 '100': 19.07744332508022,
 '101': 0.20055749351268304,
 '110': 3.2323526824973547,
 '111': 17.200241781855333}

In [61]:
# compute the variance and mean
mu = sum(output_values)/len(output_values)
var = 0

for val in output_values:
    var += ((val-mu)**2)

var = var/len(output_values)
print(mu, var)

11.883423621372799 50.114978641125255


Run algorithm

In [64]:
m = 7 # ancilla qubits
j = int(np.ceil(np.log2(max(mapping.values())))) # max intervals

k = 32 # this value is 32*B, and B is set to 1
mean_proxy = sum(random.choices(output_values, k=k))/k # mean proxy
new_output_values = [value/mean_proxy for value in output_values] # scaled output values

mu_estimation = 0
temp = []

for value in new_output_values:
    temp.append(A_2(value, 0, 1))

# create state preperation
qr = QuantumRegister(n+1)
state_prearation = QuantumCircuit(qr)

# create uniform probability distribution
state_prearation.h([x+1 for x in range(n)]) 

# apply state function
for i, amplitude in enumerate(temp):
    state_function(state=bit_strings[i], value=amplitude, qc=state_prearation)

Q = GroverOperator(oracle=oracle, 
                   state_preparation=state_prearation, 
                   zero_reflection=None, 
                   reflection_qubits=None, 
                   insert_barriers=False, 
                   mcx_mode='noancilla', 
                   name='Q')

sampler = Sampler()
ae = AmplitudeEstimation(
    num_eval_qubits=m,  # the number of evaluation qubits specifies circuit width and accuracy
    sampler=sampler,
)

problem = EstimationProblem(
    state_preparation=state_prearation,  # A operator
    grover_operator=Q,  # Q operator
    objective_qubits=[0],  # the "good" state Psi1 is identified as measuring |1> in qubit 0
)

ae_result = ae.estimate(problem)
mu_estimation += ae_result.estimation

for i in range(j):
    temp = []
    for value in new_output_values:
        temp.append(A_2(value, 2**i, 2**(i+1))/2**(i+1))

    # create state preperation
    qr = QuantumRegister(n+1)
    state_prearation = QuantumCircuit(qr)

    # create uniform probability distribution
    state_prearation.h([x+1 for x in range(n)]) 

    # apply state function
    for x, amplitude in enumerate(temp):
        state_function(state=bit_strings[x], value=amplitude, qc=state_prearation)

    Q = GroverOperator(oracle=oracle, 
                       state_preparation=state_prearation, 
                       zero_reflection=None, 
                       reflection_qubits=None, 
                       insert_barriers=False, 
                       mcx_mode='noancilla', 
                       name='Q')

    sampler = Sampler()
    ae = AmplitudeEstimation(
        num_eval_qubits=m,  # the number of evaluation qubits specifies circuit width and accuracy
        sampler=sampler,
    )

    problem = EstimationProblem(
        state_preparation=state_prearation,  # A operator
        grover_operator=Q,  # Q operator
        objective_qubits=[0],  # the "good" state Psi1 is identified as measuring |1> in qubit 0
    )

    ae_result = ae.estimate(problem)
    mu_estimation += ae_result.estimation*2**(i+1)


mu = mu_estimation*mean_proxy
print(mu)

11.98334997700838
